### Segmenting & Clustering Toronto Neighborhood

In [2]:
#Import required libraries
import numpy as np
import pandas as pd

Read the wikipedia file on Canada postal codes and neighborhood using Pandas

In [3]:
path = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
df = pd.read_html(path)[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [5]:
# clean up the dataframe to remove the rows with Borough not assigned
df = df[df['Borough'] != 'Not assigned']

For postal codes with more than one neighborhood, split the neighborhood separated by '/' and make it a list separated by commas

In [6]:
df["Neighborhood"] = df["Neighborhood"].str.split("/", expand = False)

In [7]:
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,[Parkwoods]
3,M4A,North York,[Victoria Village]
4,M5A,Downtown Toronto,"[Regent Park , Harbourfront]"
5,M6A,North York,"[Lawrence Manor , Lawrence Heights]"
6,M7A,Downtown Toronto,"[Queen's Park , Ontario Provincial Government]"


In [8]:
#check if there is any row with neighborhood "Not assigned" or "NaN"

In [9]:
(df['Neighborhood'] == 'Not assigned').value_counts()

False    103
Name: Neighborhood, dtype: int64

In [10]:
(df['Neighborhood'] == 'NaN').value_counts()

False    103
Name: Neighborhood, dtype: int64

In [11]:
#display the shape
df.shape

(103, 3)

In [12]:
url = 'http://cocl.us/Geospatial_data'
location = pd.read_csv(url)
location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
location['Postal code'] = location['Postal Code']
location.drop('Postal Code', axis=1, inplace=True)
location.head()

,Latitude,Longitude,Postal code
0,43.806686,-79.194353,M1B
1,43.784535,-79.160497,M1C
2,43.763573,-79.188711,M1E
3,43.770992,-79.216917,M1G
4,43.773136,-79.239476,M1H


In [23]:
df1 = pd.merge(df, location, on = 'Postal code')
df1.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,[Parkwoods],43.753259,-79.329656
1,M4A,North York,[Victoria Village],43.725882,-79.315572
2,M5A,Downtown Toronto,"[Regent Park , Harbourfront]",43.654260,-79.360636
3,M6A,North York,"[Lawrence Manor , Lawrence Heights]",43.718518,-79.464763
4,M7A,Downtown Toronto,"[Queen's Park , Ontario Provincial Government]",43.662301,-79.389494
